In [34]:
import getpass
import os
from langchain_groq import ChatGroq
import time
import langchain, langgraph

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("llmgsk_6QCRZZGxVnJEd9PVrxmfWGdyb3FYLMhr7XESbMf6LJeElvRKiy0C")

In [35]:
llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model_name="llama3-70b-8192")

In [ ]:
class WorkflowContext:
    def __init__(self, user, email, number):
        self.user = user
        self.email = email
        self.number = number

context = WorkflowContext(user="Alice", email="alice@example.com", number="123456789")
print(context.number)

### Nodos   

In [55]:
class WaitNode(context):

    def __init__(self, name, wait_time, on_timeout, on_tick, tick_interval):
        self.name = name
        self.wait_time = wait_time
        self.on_timeout = on_timeout
        self.on_tick = on_tick
        self.tick_interval = tick_interval
        self.last_tick = time.time()
    
    def execute(self, context):
        context = context.copy()
        time.sleep(self.wait_time)
        if self.on_timeout:
            return self.on_timeout()
        else:
            return self.on_tick()

In [53]:
class RedirectNode():
    #super().__init__(id)
    def __init__(self, id, redirected_to):
        self.id = id
        self.redirected_to = redirected_to

    def execute(self, context):
        return self.redirected_to

In [52]:
class SendMessageNode():
    def __init__(self,id, message):
        self.id = id
        self.message = message

    def execute(self, context):
        print(self.message)

In [51]:
class SurveyNode():
    def __init__(self, id, survey_questions, on_complete):
        #super().__init__(id)
        self.id = id
        self.survey_questions = survey_questions
        self.on_complete = on_complete
    
    def execute(self, context):
        for question in self.survey_questions:
            print(question)
        return self.on_complete

In [21]:
timer_5min_node = WaitNode(
    name="timer_5min_node",
    wait_time=300,
    on_timeout="redirect_to_survey",
    on_tick="send_reminder",
    tick_interval=120
)

In [22]:
send_reminder_node = SendMessageNode(
    id="send_reminder",
    message="Recuerda iniciar sesion para continuar, tienes 3 minutos restantes antes que se finalice el chat.",
)

In [23]:
redirect_to_survey_node = RedirectNode(
    id="redirect_to_survey_node",
    redirected_to="survey",
)

In [26]:
survey_node = SurveyNode(
    id="survey",
    survey_questions=get_questions(),
    on_complete="process_survey_results",
)

In [25]:
def get_questions():
    questions = [
        "¿Como calificarías la información recibida?",
        "¿Deseas dejar algun comentario?",
        
    ]
    '''if context.previous_dissatisfaction:
        questions.append("Parece que tuviste problemas anteriormente. ¿Qué podriamos mejorar?")
    '''
    return questions

In [ ]:
def SentimentalAnalysis(comment):
    prompt = (
        f"Clasifica el siguiente comentario como 'positivo', 'negativo' o 'neutral':\n"
        f"Comentario: \"{comment}\"\n"
        f"Respuesta:"
    )
    response = llm.predict(prompt)
    return response.strip().lower()

'''

# Ejemplo de uso
resultado = SentimentalAnalysis("Ni bien ni mal")
print(resultado)  # Output esperado: "positivo"

'''

In [ ]:
def process_survey_results(survey_results, context):
    comment = survey_results.get('comentarios', '')
    sentiment = SentimentalAnalysis(comment)

    whatsapp_message = (
        "Gracias por tu retroalimentación. Estamos muy contentos de que hayas tenido éxito con nuestro producto. "
        "Si tienes alguna pregunta o necesitas ayuda adicional, no dudes en contactarnos. ¡Estamos aquí para ayudarte!"
        if sentiment == "positivo"
        else f"Lamentamos que no estés satisfecho. {survey_results.get('comentarios_adicionales', '')}"
    )

    print(whatsapp_message)
'''
implementacion envio de mensajes
    try:
        send_whatsapp_message(context.number, whatsapp_message)
    except Exception as e:
        log_error(f"Error enviando mensaje por WhatsApp: {e}")
'''

In [ ]:
StartNode: {
    print("Hola, ¿en qué puedo ayudarte hoy?")
}

EndNode: {
    print("¡Gracias por usar el chatbot!")
}

### Se debe crear una funcion en twllio para enviar mensajes a los usuarios

## Ejemplo:
def send_whatsapp_message(phone_number, message):

    client = Client(account_sid, auth_token)

    try:
        message = client.messages.create(
            body=message,
            from_=twilio_number,
            to=f'whatsapp:{phone_number}'
        )
        print(f"Mensaje enviado a {phone_number}: {message.sid}")
    except Exception as e:
        print(f"Error al enviar el mensaje: {e}")

In [64]:
def start(input):
    return input + "Bienvenido al chatbot de prueba"

def end(input):
    input = input
    return "Gracias por usar el chatbot"

In [ ]:
import time

class WaitNode:
    def __init__(self, name, wait_time, on_timeout, on_tick, tick_interval):
        self.name = name
        self.wait_time = wait_time
        self.on_timeout = on_timeout
        self.on_tick = on_tick
        self.tick_interval = tick_interval
        self.last_tick = time.time()

    def execute(self):
        start_time = time.time()
        while time.time() - start_time < self.wait_time:
            time.sleep(self.tick_interval)
            if self.on_tick:
                self.on_tick()
            self.last_tick = time.time()
        
        if self.on_timeout:
            return self.on_timeout()
        return None

def redirect_to_survey():
    return "Redirigiendo a la encuesta."

def send_reminder():
    print("Enviando recordatorio.")

# Crear el nodo de espera
timer_5min_node = WaitNode(
    name="timer_5min_node",
    wait_time=300,  # 5 minutos
    on_timeout=redirect_to_survey,
    on_tick=send_reminder,
    tick_interval=120  # 2 minutos
)

# Ejecutar el nodo de espera
result = timer_5min_node.execute()
print(result)


In [66]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("node_1", start)
workflow.add_node("node_2", )
workflow.add_node("node_5", end)

workflow.add_edge("node_1", "node_2")

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [67]:
app.invoke("hola mundo")


'Gracias por usar el chatbot'